In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#importing the neccessary libraries
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, StandardScaler,PolynomialFeatures
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score, train_test_split, KFold, StratifiedShuffleSplit, GridSearchCV

In [3]:
df=pd.read_csv('/content/drive/MyDrive/Colab resources/predictive analysis/train.csv')


In [4]:
df.head(5)
#this tells us that we have about 81 columns

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [5]:
# Drop columns with high null value count
high_null_count_features = df.columns[df.isnull().sum() >= 1000]
df.drop(columns=high_null_count_features, inplace=True)


In [6]:
high_null_count_features

Index(['Alley', 'PoolQC', 'Fence', 'MiscFeature'], dtype='object')

In [7]:
# Fill missing values using simple imputer from sklearn
from sklearn.impute import SimpleImputer
null_features = df.columns[df.isnull().sum() > 0]
for feature in null_features:
    if df[feature].dtype == "object":
        imputer = SimpleImputer(strategy="most_frequent")
    else:
        imputer = SimpleImputer(strategy="mean")
    df[feature] = imputer.fit_transform(df[feature].values.reshape(-1, 1)).ravel()


In [8]:
#Using Ordinal encoder for encoding
encoders = {}
for feature in df.columns:
    if df[feature].dtype == "object":
        encoder = OrdinalEncoder()
        df[feature] = encoder.fit_transform(df[feature].values.reshape(-1, 1)).ravel()
        encoders[feature] = encoder

In [9]:
#using correaltion to find the best features
correlation_saleprice = df.corr()['SalePrice'].sort_values(ascending=False)
top_features = correlation_saleprice.index[:40]
correlation_matrix = df[top_features].corr()

In [10]:
# Remove 'SalePrice' from correlations to avoid self-correlation
correlation_saleprice = correlation_saleprice.drop('SalePrice')

# Selecting top correlated features
top_features = correlation_saleprice.index[:40]

# Print summary of top correlated features with their correlations
print("Top Correlated Features with SalePrice:")
for feature in top_features:
    correlation_value = correlation_saleprice[feature]
    print(f"{feature}: {correlation_value:.2f}")
print(top_features)

Top Correlated Features with SalePrice:
OverallQual: 0.79
GrLivArea: 0.71
GarageCars: 0.64
GarageArea: 0.62
TotalBsmtSF: 0.61
1stFlrSF: 0.61
FullBath: 0.56
TotRmsAbvGrd: 0.53
YearBuilt: 0.52
YearRemodAdd: 0.51
MasVnrArea: 0.48
GarageYrBlt: 0.47
Fireplaces: 0.47
BsmtFinSF1: 0.39
Foundation: 0.38
LotFrontage: 0.33
WoodDeckSF: 0.32
2ndFlrSF: 0.32
OpenPorchSF: 0.32
HalfBath: 0.28
LotArea: 0.26
CentralAir: 0.25
Electrical: 0.23
PavedDrive: 0.23
BsmtFullBath: 0.23
RoofStyle: 0.22
BsmtUnfSF: 0.21
SaleCondition: 0.21
Neighborhood: 0.21
HouseStyle: 0.18
BedroomAbvGr: 0.17
GarageCond: 0.13
RoofMatl: 0.13
ExterCond: 0.12
Functional: 0.12
ScreenPorch: 0.11
Exterior2nd: 0.10
Exterior1st: 0.10
GarageQual: 0.10
PoolArea: 0.09
Index(['OverallQual', 'GrLivArea', 'GarageCars', 'GarageArea', 'TotalBsmtSF',
       '1stFlrSF', 'FullBath', 'TotRmsAbvGrd', 'YearBuilt', 'YearRemodAdd',
       'MasVnrArea', 'GarageYrBlt', 'Fireplaces', 'BsmtFinSF1', 'Foundation',
       'LotFrontage', 'WoodDeckSF', '2ndFlrSF',

In [11]:
best_features=['OverallQual', 'GrLivArea', 'GarageCars', 'GarageArea', 'TotalBsmtSF',
       '1stFlrSF', 'FullBath', 'TotRmsAbvGrd', 'YearBuilt', 'YearRemodAdd',
       'MasVnrArea', 'GarageYrBlt', 'Fireplaces', 'BsmtFinSF1', 'Foundation',
       'LotFrontage', 'WoodDeckSF', '2ndFlrSF', 'OpenPorchSF', 'HalfBath',
       'LotArea', 'CentralAir', 'Electrical', 'PavedDrive', 'BsmtFullBath',
       'RoofStyle', 'BsmtUnfSF', 'SaleCondition', 'Neighborhood', 'HouseStyle',
       'BedroomAbvGr', 'GarageCond', 'RoofMatl', 'ExterCond', 'Functional',
       'ScreenPorch', 'Exterior2nd', 'Exterior1st', 'GarageQual', 'PoolArea']

In [12]:
bf=df[best_features]

In [13]:
bf

,OverallQual,GrLivArea,GarageCars,GarageArea,TotalBsmtSF,1stFlrSF,FullBath,TotRmsAbvGrd,YearBuilt,YearRemodAdd,...,BedroomAbvGr,GarageCond,RoofMatl,ExterCond,Functional,ScreenPorch,Exterior2nd,Exterior1st,GarageQual,PoolArea
0,7,1710,2,548,856,856,2,8,2003,2003,...,3,4.0,1.0,4.0,6.0,0,13.0,12.0,4.0,0
1,6,1262,2,460,1262,1262,2,6,1976,1976,...,3,4.0,1.0,4.0,6.0,0,8.0,8.0,4.0,0
2,7,1786,2,608,920,920,2,6,2001,2002,...,3,4.0,1.0,4.0,6.0,0,13.0,12.0,4.0,0
3,7,1717,3,642,756,961,1,7,1915,1970,...,3,4.0,1.0,4.0,6.0,0,15.0,13.0,4.0,0
4,8,2198,3,836,1145,1145,2,9,2000,2000,...,4,4.0,1.0,4.0,6.0,0,13.0,12.0,4.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,6,1647,2,460,953,953,2,7,1999,2000,...,3,4.0,1.0,4.0,6.0,0,13.0,12.0,4.0,0
1456,6,2073,2,500,1542,2073,2,7,1978,1988,...,3,4.0,1.0,4.0,2.0,0,10.0,9.0,4.0,0
1457,7,2340,1,252,1152,1188,2,9,1941,2006,...,4,4.0,1.0,2.0,6.0,0,5.0,5.0,4.0,0
1458,5,1078,1,240,1078,1078,1,5,1950,1996,...,2,4.0,1.0,4.0,6.0,0,8.0,8.0,4.0,0


In [14]:
# Separating the target column ('SalePrice')
Y_data = df['SalePrice']
X_data = df.drop(columns=['SalePrice'])

# Splitting data after preprocessing
X_processed = X_data.copy()
y_processed = Y_data.copy()

# Splitting data into train and test sets after preprocessing
X_train_processed, X_test_processed, y_train_processed, y_test_processed = train_test_split(
    X_processed, y_processed, test_size=0.2, random_state=42
)

# Scaling the features using StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_data)

# Define random seed
random_seed = 42

# Splitting the dataset into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(
    X_scaled, Y_data,
    shuffle=True,
    random_state=random_seed,
    train_size=0.7,
    test_size=0.3
)

In [15]:
#A way of putting all the models in a loop and executing them one by one
models = [
    LinearRegression(),
    DecisionTreeRegressor(max_depth=5),
    RandomForestRegressor()
]

for model in models:
  model.fit(x_train, y_train)
  train_pred = model.predict(x_train)
  test_pred = model.predict(x_test)
  train_mse, train_mae, train_r2 = mean_squared_error(y_train, train_pred), mean_absolute_error(y_train, train_pred), r2_score(y_train, train_pred)
  test_mse, test_mae, test_r2 = mean_squared_error(y_test, test_pred), mean_absolute_error(y_test, test_pred), r2_score(y_test, test_pred)
  model_name = type(model).__name__
  print(f"Model Name: {model_name}")
  print(f"Train MSE: {train_mse}, Test MSE: {test_mse}")
  print(f"Train MAE: {train_mae}, Test MAE: {test_mae}")
  print(f"Train R2: {train_r2}, Test R2: {test_r2}")

Model Name: LinearRegression
Train MSE: 945644342.4638679, Test MSE: 1084194169.678348
Train MAE: 18837.741075842143, Test MAE: 21968.244126320373
Train R2: 0.8430314148561591, Test R2: 0.8446288905220432
Model Name: DecisionTreeRegressor
Train MSE: 895081049.5755715, Test MSE: 1570997960.53916
Train MAE: 21568.453524522763, Test MAE: 26908.277280541708
Train R2: 0.8514244736294079, Test R2: 0.7748671751398635
Model Name: RandomForestRegressor
Train MSE: 129961066.12717275, Test MSE: 733922405.3210491
Train MAE: 6710.546777668952, Test MAE: 17047.8149086758
Train R2: 0.9784276141063606, Test R2: 0.894824800229933


In [16]:
#Preparing test data for prediction
test=pd.read_csv('/content/drive/MyDrive/Colab resources/predictive analysis/test.csv')
test.drop(['Alley', 'PoolQC', 'Fence', 'MiscFeature'],axis=1,inplace=True)

In [17]:

null_features = test.columns[test.isnull().sum() > 0]
for feature in null_features:
    if test[feature].dtype == "object":
        imputer = SimpleImputer(strategy="most_frequent")
    else:
        imputer = SimpleImputer(strategy="mean")
    test[feature] = imputer.fit_transform(test[feature].values.reshape(-1, 1)).ravel()


In [18]:
encoders = {}
for feature in test.columns:
    if test[feature].dtype == "object":
        encoder = OrdinalEncoder()
        test[feature] = encoder.fit_transform(test[feature].values.reshape(-1, 1)).ravel()
        encoders[feature] = encoder

In [19]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(test)
for model in models:
  result=model.predict(X_scaled)
  print(type(model).__name__)
  print(result)

LinearRegression
[ 4.26708781e+15 -1.20441601e+15  5.44823919e+14 ...  1.36200699e+14
  1.56715861e+15 -9.04991898e+14]
DecisionTreeRegressor
[121273.45731707 147159.07638889 208470.         ... 147159.07638889
 121273.45731707 192700.        ]
RandomForestRegressor
[127842.   154400.82 192616.5  ... 156175.7  117016.   227630.9 ]


In [20]:
#collecting the results of randomforest
test['price']=model.predict(X_scaled)

In [21]:
dummy=pd.read_csv('/content/drive/MyDrive/Colab resources/predictive analysis/test.csv')

In [22]:
#putting the results in to a csv file
dummy['price']=test['price']
dummy.to_csv('/content/drive/MyDrive/Colab resources/predictive analysis/house_results.csv',index=True)

In [23]:
#using half the columns best in correlation and matching it to almost same r2 score infact better

scaler = StandardScaler()
bf = scaler.fit_transform(bf)

# Splitting the dataset into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(
    bf, Y_data,
    shuffle=True,
    random_state=random_seed,
    train_size=0.7,
    test_size=0.3
)


In [24]:
# This is the process where the best features are put into randomforest for processing the results
reg=RandomForestRegressor()
reg.fit(x_train,y_train)
test_pred=reg.predict(x_test)
r2=r2_score(y_test, test_pred)
print(r2)

0.8945475094147297


In [25]:
#This is the polynomial regression results
poly_reg = PolynomialFeatures(degree = 4)
X_poly = poly_reg.fit_transform(x_train)
model = LinearRegression()
model.fit(X_poly, y_train)

train_pred=model.predict(X_poly)
x_p = poly_reg.fit_transform(x_test)
test_pred=model.predict(x_p)
train_mse, train_mae, train_r2 = mean_squared_error(y_train, train_pred), mean_absolute_error(y_train, train_pred), r2_score(y_train, train_pred)
test_mse, test_mae, test_r2 = mean_squared_error(y_test, test_pred), mean_absolute_error(y_test, test_pred), r2_score(y_test, test_pred)
print("this is polynomial regression")
print("the train_mse is:",train_mse)
print("the train_mae is ",train_mae)
print("the train_r2",train_r2)
print("the test_mse is:",test_mse)
print("the test_mae is ",test_mae)
print("the test_r2",test_r2)

this is polynomial regression
the train_mse is: 33736.532811005956
the train_mae is  8.12946887748055
the train_r2 0.9999944000343626
the test_mse is: 9.19331475221869e+21
the test_mae is  7022657033.857362
the test_r2 -1317453600820.3794
